In [8]:
import ollama
import json

# Initialize the Ollama client
client = ollama.Client()

# Define some simple functions
def get_weather(city):
    """Get the weather for a specific city"""
    # Simulate weather data
    weather_data = {
        "New York": "Sunny, 72°F",
        "London": "Rainy, 15°C", 
        "Tokyo": "Cloudy, 25°C",
        "Paris": "Partly cloudy, 18°C"
    }
    return weather_data.get(city, f"Weather data not available for {city}")

def calculate_tip(bill_amount, tip_percentage=15):
    """Calculate tip amount"""
    tip = (bill_amount * tip_percentage) / 100
    total = bill_amount + tip
    return f"Bill: ${bill_amount}, Tip ({tip_percentage}%): ${tip:.2f}, Total: ${total:.2f}"

# Available functions mapping
available_functions = {
    "get_weather": get_weather,
    "calculate_tip": calculate_tip
}

# Define tools for the model
tools = [
    {
        'type': 'function',
        'function': {
            'name': 'get_weather',
            'description': 'Get current weather information for a city',
            'parameters': {
                'type': 'object',
                'properties': {
                    'city': {
                        'type': 'string',
                        'description': 'The city name to get weather for'
                    }
                },
                'required': ['city']
            }
        }
    },
    {
        'type': 'function',
        'function': {
            'name': 'calculate_tip',
            'description': 'Calculate tip and total amount for a bill',
            'parameters': {
                'type': 'object',
                'properties': {
                    'bill_amount': {
                        'type': 'number',
                        'description': 'The bill amount in dollars'
                    },
                    'tip_percentage': {
                        'type': 'number',
                        'description': 'Tip percentage (default: 15)',
                        'default': 15
                    }
                },
                'required': ['bill_amount']
            }
        }
    }
]

# User message
user_message = "What's the weather in Tokyo? Also, calculate tip for a $50 bill with 20% tip."

# Conversation history
messages = [
    {'role': 'user', 'content': user_message}
]

print(f"User: {user_message}\n")

# First call: Let the model decide which tools to use
response = client.chat(
    model='llama3.1:latest',
    messages=messages,
    tools=tools
)

# Check if the model wants to call any functions
if response['message'].get('tool_calls'):
    print("🤖 AI is calling functions...\n")
    
    # Add the AI's response to conversation
    messages.append(response['message'])
    
    # Process each function call
    for call in response['message']['tool_calls']:
        function_name = call['function']['name']
        
        # Handle arguments (could be string or dict)
        if isinstance(call['function']['arguments'], str):
            function_args = json.loads(call['function']['arguments'])
        else:
            function_args = call['function']['arguments']
        
        print(f"📞 Calling: {function_name}({function_args})")
        
        # Call the actual function
        function_to_call = available_functions[function_name]
        function_response = function_to_call(**function_args)
        
        print(f"📋 Result: {function_response}\n")
        
        # Add function result to conversation
        messages.append({
            'role': 'tool',
            'content': function_response,
        })

# Second call: Get the final response
final_response = client.chat(
    model='llama3.1:latest',
    messages=messages
)

print(f"🤖 AI Assistant: {final_response['message']['content']}")

User: What's the weather in Tokyo? Also, calculate tip for a $50 bill with 20% tip.

🤖 AI is calling functions...

📞 Calling: get_weather({'city': 'Tokyo'})
📋 Result: Cloudy, 25°C

📞 Calling: calculate_tip({'bill_amount': 50, 'tip_percentage': 20})
📋 Result: Bill: $50, Tip (20%): $10.00, Total: $60.00

🤖 AI Assistant: Please note that the weather forecast is subject to change and might not be up-to-date. For the most accurate and current information, I recommend checking a reliable weather website or app.

Also, please note that tipping customs may vary in different regions, and it's always a good idea to check local norms before leaving a tip.
